# Soft SVM (Support vector machine) using convex Quadratic program 

## Step 1: Import All required Libraries

In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

## Step 2: Read CSV file (For this i have uploded the file on google colab) 

In [27]:
df = pd.read_csv('file.csv')
print(df.shape,df)

(2665, 6)       Temperature   Humidity       Light          CO2  HumidityRatio  Occupancy
0       23.700000  26.272000  585.200000   749.200000       0.004764          1
1       23.718000  26.290000  578.400000   760.400000       0.004773          1
2       23.730000  26.230000  572.666667   769.666667       0.004765          1
3       23.722500  26.125000  493.750000   774.750000       0.004744          1
4       23.754000  26.200000  488.600000   779.000000       0.004767          1
...           ...        ...         ...          ...            ...        ...
2660    24.290000  25.700000  808.000000  1150.250000       0.004829          1
2661    24.330000  25.736000  809.800000  1129.200000       0.004848          1
2662    24.330000  25.700000  817.000000  1125.800000       0.004841          1
2663    24.356667  25.700000  813.000000  1123.000000       0.004849          1
2664    24.408333  25.681667  798.000000  1124.000000       0.004860          1

[2665 rows x 6 columns]


## Step 3: Store the features and target in X , Y seperately and since target has value 0,1 change label 0 to -1 

In [28]:
X = df.iloc[:,:-1].values
Y = df.iloc[:,-1:].values
print(X.shape,X)
Y[Y==0]=-1
print(Y.shape,Y)

(2665, 5) [[2.37000000e+01 2.62720000e+01 5.85200000e+02 7.49200000e+02
  4.76416302e-03]
 [2.37180000e+01 2.62900000e+01 5.78400000e+02 7.60400000e+02
  4.77266099e-03]
 [2.37300000e+01 2.62300000e+01 5.72666667e+02 7.69666667e+02
  4.76515255e-03]
 ...
 [2.43300000e+01 2.57000000e+01 8.17000000e+02 1.12580000e+03
  4.84075873e-03]
 [2.43566667e+01 2.57000000e+01 8.13000000e+02 1.12300000e+03
  4.84855928e-03]
 [2.44083333e+01 2.56816667e+01 7.98000000e+02 1.12400000e+03
  4.86020770e-03]]
(2665, 1) [[1]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]


## Step 4: Processing the Data and Splitting


In [29]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)
X_scale

array([[0.83168317, 0.44513204, 0.34479305, 0.33003334, 0.70421197],
       [0.8359604 , 0.44705255, 0.34078657, 0.34152347, 0.70830847],
       [0.83881188, 0.44065084, 0.33740855, 0.35103018, 0.70468898],
       ...,
       [0.98138614, 0.38410243, 0.48136692, 0.71638882, 0.74113545],
       [0.98772277, 0.38410243, 0.47901016, 0.71351629, 0.74489576],
       [1.        , 0.38214635, 0.47017234, 0.71454219, 0.75051096]])

A. Split the data in 70,30 for training and testing the model 

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_scale, Y, test_size=0.3, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1865, 5) (800, 5) (1865, 1) (800, 1)


B. Split the data in 80,20 for training and testing the model 

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2132, 5) (533, 5) (2132, 1) (533, 1)


C. Split the data in 90,10 for training and testing the model 

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2398, 5) (267, 5) (2398, 1) (267, 1)


## Step 5: Optimizing the weight vector w using the cvxopt_solvers
## Here we need to change the variables as:
### max(a) 1/2*a^T*H^T*a - 1^T*a
### such that -a<=0 and y^T*a=0
### where for the Quadratic solver the respective variables are:
### X=a , P=H (matrix of size m×m), q=-1 ( a vector of size m×1), G=-a (matrix of size 2m×m, such that a diagonal matrix of -1s of size m×m is concatenated vertically with another diagonal matrix of 1s of size m×m) , h=0 (a vector of size 2m×1, with zeros in the first m cells and C in the other m cells) , A=y (the label vector of size m×1) , b=0 , H = ghram matrix
## Also for regularization c=10 is used.

In [33]:
#Initializing values and computing H. Note the 1.0 to force to float type
c=10
samples,features = X_train.shape
y = y_train.reshape(-1,1) * 1.0
X_dash = y*X_train
H = np.dot(X_dash , X_dash.T) * 1.0

#Converting into cvxopt format
P = cvxopt_matrix(H)
q = cvxopt_matrix(-np.ones((samples, 1)))
G = cvxopt_matrix(np.vstack((np.eye(samples)*-1,np.eye(samples))))
h = cvxopt_matrix(np.hstack((np.zeros(samples), np.ones(samples)))*c)
A = cvxopt_matrix(y.reshape(1, -1))
b = cvxopt_matrix(np.zeros(1))

## Step 6: Calculating optimized value of w using the defined solver and checking for alphas. 

In [34]:
sol = cvxopt_solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

     pcost       dcost       gap    pres   dres
 0: -1.4534e+03 -5.5336e+05  4e+06  3e+00  7e-08
 1: -1.1295e+03 -3.4788e+05  6e+05  3e-01  6e-08
 2: -8.6375e+02 -5.4004e+04  7e+04  2e-02  5e-08
 3: -8.4338e+02 -3.7391e+04  5e+04  1e-02  5e-08
 4: -8.5219e+02 -1.4800e+04  2e+04  4e-03  5e-08
 5: -9.4221e+02 -5.5628e+03  5e+03  8e-04  6e-08
 6: -9.9280e+02 -2.1240e+03  1e+03  2e-04  6e-08
 7: -1.0082e+03 -2.1331e+03  1e+03  2e-04  6e-08
 8: -9.9800e+02 -2.0541e+03  1e+03  1e-04  5e-08
 9: -1.0056e+03 -1.8043e+03  8e+02  5e-05  5e-08
10: -1.0193e+03 -1.8131e+03  8e+02  5e-05  5e-08
11: -1.0284e+03 -1.4854e+03  5e+02  2e-05  5e-08
12: -1.0370e+03 -1.4928e+03  5e+02  2e-05  5e-08
13: -1.0266e+03 -1.4549e+03  4e+02  1e-05  5e-08
14: -1.0318e+03 -1.3898e+03  4e+02  8e-06  5e-08
15: -1.0297e+03 -1.3853e+03  4e+02  7e-06  5e-08
16: -1.0432e+03 -1.3847e+03  3e+02  4e-06  5e-08
17: -1.0519e+03 -1.2056e+03  2e+02  1e-06  5e-08
18: -1.0563e+03 -1.2098e+03  2e+02  9e-07  5e-08
19: -1.0507e+03 -1.20

## Here W = alphai*yi*xi
## Also b = 1/n(sum(y[i]-W*xi))

In [35]:
# w parameters calculated using the alphas which we get after the solver gives output.
w = np.dot((y_train * alphas).T,X_train).reshape(-1,1)

# Computing b
is_sv = (alphas > 1e-4).flatten()
b = y[is_sv] - np.dot(X_train[is_sv], w)
b = sum(b)/len(b)

# Display results
print('Alphas = ',alphas[alphas > 1e-4])
print('w = ', w.flatten())
print('b = ', b)

Alphas =  [9.99999477 9.99999998 9.99998669 9.99999859 9.99999999 9.99999999
 9.99999937 9.99999998 9.99999995 9.99999893 9.99999612 9.99999999
 9.99999998 9.99999998 9.99999999 9.99999998 9.99999998 9.99999923
 9.99999998 9.99999998 9.99999877 9.99999999 9.9999991  9.99999958
 9.99999998 9.99999784 9.99999937 9.99999998 9.99999998 9.99999816
 9.99999902 9.99999999 9.99999999 9.99999065 9.99999859 9.99999894
 9.99999932 9.99999999 9.99999999 9.99999998 9.99999998 9.9999989
 9.99999998 9.99999615 9.99999931 9.99999398 9.99999756 9.99999998
 9.99999998 9.99999999 9.99999998 9.99999914 9.99999999 9.99999998
 9.99999998 9.9999995  7.06180829 9.99999998 9.99999949 9.99999895
 9.99999959 9.44546083 9.9999989  9.99999937 9.99999828 8.98581196
 9.99999878 9.99999844 9.99997247 9.99997743 9.99999824 9.99999997
 9.99999999 9.99999934 9.99999998 9.99999998 9.99999998 9.99999999
 9.99999999 9.99999937 9.99995924 9.99999998 9.99999813 9.99999862
 9.99999933 9.99999999 9.99999999 9.99999964 5.615815

In [36]:
X_Support_vectors = X_train[is_sv]
y_support_vectors = y[is_sv]
print(alphas,'\n',"Support vectors are : = ",X_Support_vectors,'\n','\n',"Number of support vectors = ",len(X_Support_vectors))

[[4.43688822e-08]
 [3.49668905e-07]
 [2.79959566e-08]
 ...
 [3.40195086e-07]
 [2.67551743e-08]
 [2.85867082e-07]] 
 Support vectors are : =  [[2.25600000e+01 2.50200000e+01 4.33000000e+02 8.30750000e+02
  4.23128418e-03]
 [2.25400000e+01 2.51600000e+01 4.27400000e+02 8.53600000e+02
  4.24992107e-03]
 [2.27000000e+01 3.12900000e+01 4.33000000e+02 1.39250000e+03
  5.34630672e-03]
 [2.26000000e+01 2.50000000e+01 4.33000000e+02 8.37000000e+02
  4.23823119e-03]
 [2.22000000e+01 2.60000000e+01 6.08250000e+02 9.46000000e+02
  4.30227873e-03]
 [2.12000000e+01 2.52900000e+01 4.64000000e+02 7.77750000e+02
  3.93429348e-03]
 [2.23900000e+01 2.47170833e+01 4.19000000e+02 7.72925000e+02
  4.13646442e-03]
 [2.24175000e+01 2.47225000e+01 4.19000000e+02 7.89750000e+02
  4.14434755e-03]
 [2.03900000e+01 2.37000000e+01 3.11750000e+02 4.74750000e+02
  3.50518545e-03]
 [2.26000000e+01 2.51666667e+01 4.33000000e+02 8.54333333e+02
  4.26667988e-03]
 [2.26000000e+01 2.50000000e+01 4.33000000e+02 8.32000000e+

## Step 7: Creating target Array and calculating the target values.

In [37]:
print(X_test.shape,w.shape)
target = np.where(np.dot(X_test,w)+b>=0.0,1,-1)
t = np.where(np.dot(X_train,w)+b>=0.0,1,-1)
print(target.flatten(),'\n',t.flatten(),'\n',y_test.flatten())

(267, 5) (5, 1)
[ 1 -1 -1 -1 -1 -1 -1  1  1  1 -1  1 -1 -1 -1 -1  1  1  1 -1  1 -1  1 -1
 -1  1 -1 -1  1 -1  1 -1  1  1  1  1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1
  1 -1 -1 -1  1 -1  1  1 -1  1 -1  1  1 -1 -1 -1  1 -1 -1  1 -1 -1  1  1
 -1 -1  1 -1 -1  1  1 -1 -1  1 -1  1 -1 -1 -1 -1  1 -1  1 -1 -1 -1  1 -1
 -1  1 -1 -1 -1 -1 -1 -1 -1 -1  1  1 -1 -1  1  1  1  1 -1 -1 -1  1 -1 -1
 -1 -1  1  1 -1 -1 -1 -1 -1  1 -1  1 -1 -1  1 -1 -1 -1  1 -1  1 -1 -1  1
  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1  1 -1 -1 -1 -1 -1  1  1  1 -1
 -1 -1 -1  1  1 -1 -1 -1 -1 -1 -1  1 -1  1 -1 -1  1 -1  1 -1  1 -1  1 -1
  1 -1  1  1 -1 -1  1  1 -1  1  1  1 -1  1  1  1 -1 -1  1  1 -1  1 -1  1
  1 -1  1  1 -1 -1  1 -1  1 -1 -1  1  1  1 -1 -1  1 -1 -1 -1 -1 -1 -1  1
 -1  1 -1 -1 -1  1  1 -1 -1  1 -1 -1 -1 -1 -1  1 -1 -1 -1  1 -1 -1  1 -1
 -1 -1  1] 
 [-1  1 -1 ...  1 -1  1] 
 [ 1 -1 -1 -1 -1 -1 -1  1  1  1 -1  1 -1 -1 -1 -1  1  1  1 -1  1 -1  1 -1
 -1  1 -1 -1  1 -1  1 -1  1  1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1

## Step 8: Checking for Accuracy. 

In [38]:
v1 = 0  
v2 = 0
c = 0
i=0    
for c in range( np.size( target ) ) :  
  if y_test[c] == target[c] :            
    v1 = v1 + 1
for i in range( np.size( t ) ) :
  if y_train[i] == t[i] :            
    v2 = v2 + 1
  i = i + 1
print("Accuracy of Defined Model on test data :",(v1/c)*100)     
print("Accuracy of Defined Model on train data:",(v2/i)*100)   

Accuracy of Defined Model on test data : 97.74436090225564
Accuracy of Defined Model on train data: 98.41534612176814
